## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Sao Filipe,CV,14.90,-24.50,73.78,80,13,7.94,few clouds
1,1,Albany,US,42.60,-73.97,84.45,42,30,6.46,scattered clouds
2,2,Kavieng,PG,-2.57,150.80,81.68,78,96,15.70,light rain
3,3,Carnarvon,AU,-24.87,113.63,59.72,46,0,18.97,clear sky
4,4,Harlingen,US,26.19,-97.70,95.09,83,71,14.41,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                        (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Albany,US,42.60,-73.97,84.45,42,30,6.46,scattered clouds
2,2,Kavieng,PG,-2.57,150.80,81.68,78,96,15.70,light rain
5,5,Cayenne,GF,4.93,-52.33,81.09,1,0,1.14,clear sky
7,7,Atuona,PF,-9.80,-139.03,76.71,69,8,15.41,clear sky
10,10,Tabou,CI,4.42,-7.35,77.16,78,95,8.84,overcast clouds
15,15,Puerto Ayora,EC,-0.74,-90.35,78.76,83,57,8.01,broken clouds
16,16,Palmer,US,42.16,-72.33,83.55,41,20,0.00,few clouds
18,18,Ushibuka,JP,32.19,130.02,77.54,86,4,3.80,clear sky
19,19,Sonari,IN,27.07,95.03,75.13,98,100,2.55,moderate rain
20,20,Tevaitoa,PF,-16.78,-151.50,78.96,74,20,23.69,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cities_df = preferred_cities_df.dropna()
cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Albany,US,42.60,-73.97,84.45,42,30,6.46,scattered clouds
2,2,Kavieng,PG,-2.57,150.80,81.68,78,96,15.70,light rain
5,5,Cayenne,GF,4.93,-52.33,81.09,1,0,1.14,clear sky
7,7,Atuona,PF,-9.80,-139.03,76.71,69,8,15.41,clear sky
10,10,Tabou,CI,4.42,-7.35,77.16,78,95,8.84,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
698,698,Mangrol,IN,21.12,70.12,83.53,81,100,19.24,overcast clouds
707,707,Mongo,TD,12.18,18.69,79.20,68,69,2.80,broken clouds
708,708,Fairbanks,US,64.84,-147.72,77.59,43,75,0.00,smoke
710,710,Axim,GH,4.87,-2.24,75.96,85,100,9.08,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,84.45,scattered clouds,42.60,-73.97,
2,Kavieng,PG,81.68,light rain,-2.57,150.80,
5,Cayenne,GF,81.09,clear sky,4.93,-52.33,
7,Atuona,PF,76.71,clear sky,-9.80,-139.03,
10,Tabou,CI,77.16,overcast clouds,4.42,-7.35,
15,Puerto Ayora,EC,78.76,broken clouds,-0.74,-90.35,
16,Palmer,US,83.55,few clouds,42.16,-72.33,
18,Ushibuka,JP,77.54,clear sky,32.19,130.02,
19,Sonari,IN,75.13,moderate rain,27.07,95.03,
20,Tevaitoa,PF,78.96,light rain,-16.78,-151.50,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,84.45,scattered clouds,42.60,-73.97,
2,Kavieng,PG,81.68,light rain,-2.57,150.80,Nusa Island Retreat
5,Cayenne,GF,81.09,clear sky,4.93,-52.33,Hôtel Le Dronmi
7,Atuona,PF,76.71,clear sky,-9.80,-139.03,Villa Enata
10,Tabou,CI,77.16,overcast clouds,4.42,-7.35,hotêl doufoulougou
15,Puerto Ayora,EC,78.76,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
16,Palmer,US,83.55,few clouds,42.16,-72.33,Wedgewood Motel
18,Ushibuka,JP,77.54,clear sky,32.19,130.02,Hotei
19,Sonari,IN,75.13,moderate rain,27.07,95.03,Hotel Green View
20,Tevaitoa,PF,78.96,light rain,-16.78,-151.50,Hôtel Raiatea Lodge


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace('', np.nan, regex=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,84.45,scattered clouds,42.60,-73.97,NaN
2,Kavieng,PG,81.68,light rain,-2.57,150.80,Nusa Island Retreat
5,Cayenne,GF,81.09,clear sky,4.93,-52.33,Hôtel Le Dronmi
7,Atuona,PF,76.71,clear sky,-9.80,-139.03,Villa Enata
10,Tabou,CI,77.16,overcast clouds,4.42,-7.35,hotêl doufoulougou
...,...,...,...,...,...,...,...
698,Mangrol,IN,83.53,overcast clouds,21.12,70.12,Hotel The Royal Honours
707,Mongo,TD,79.20,broken clouds,12.18,18.69,البيت مونقو
708,Fairbanks,US,77.59,smoke,64.84,-147.72,Westmark Fairbanks Hotel & Conference Center
710,Axim,GH,75.96,overcast clouds,4.87,-2.24,Axim Beach Resort And Spa


In [13]:
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kavieng,PG,81.68,light rain,-2.57,150.80,Nusa Island Retreat
5,Cayenne,GF,81.09,clear sky,4.93,-52.33,Hôtel Le Dronmi
7,Atuona,PF,76.71,clear sky,-9.80,-139.03,Villa Enata
10,Tabou,CI,77.16,overcast clouds,4.42,-7.35,hotêl doufoulougou
15,Puerto Ayora,EC,78.76,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
698,Mangrol,IN,83.53,overcast clouds,21.12,70.12,Hotel The Royal Honours
707,Mongo,TD,79.20,broken clouds,12.18,18.69,البيت مونقو
708,Fairbanks,US,77.59,smoke,64.84,-147.72,Westmark Fairbanks Hotel & Conference Center
710,Axim,GH,75.96,overcast clouds,4.87,-2.24,Axim Beach Resort And Spa


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))